In [2]:
!pip install spacy_transformers
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 30.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [4]:
spacy.__version__

'3.7.4'

In [5]:
!nvidia-smi

Sat Mar  2 05:29:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6
Receiving objects: 100% (82/82), 5.62 MiB | 18.55 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [7]:
cv_data = json.load(open("/content/CV-Parsing-using-Spacy-3/data/training/train_data.json", "r"))

In [8]:
len(cv_data)

200

In [9]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [10]:
cv_data

[['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex

In [11]:
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False

      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break

      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label = label, alignment_mode = 'strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + "            " + str(text) + "\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [12]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size = 0.3)

In [13]:
len(train), len(test)

(140, 60)

In [14]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 60/60 [00:00<00:00, 82.53it/s]


In [15]:
len(db.tokens)

60

In [16]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 152kB/s]
config.json: 100% 481/481 [00:00<00:00, 2.79MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 12.2MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 2.34MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 31.6MB/s]
model.safetensors: 100% 499M/499M [00:01<00:00, 252MB/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P 

In [22]:
nlp = spacy.load('/content/output/model-best')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [24]:
!pip install PyMuPDF

In [25]:
!pip install python-magic
!pip install python-magic-bin
!pip install PyMuPDF
!pip install python-docx


ERROR: Could not find a version that satisfies the requirement python-magic-bin (from versions: none)
ERROR: No matching distribution found for python-magic-bin


In [84]:
import magic
import sys, fitz
from docx import Document

def identify_file_type(file_path):
    mime = magic.Magic()

    file_type = mime.from_file(file_path)

    if 'pdf' in file_type.lower():
        return 'PDF'
    else:
        return 'DOCX'

file_path = "/content/content/CV-Parsing-using-Spacy-3/data/test/resume-4.pdf"

file_format = identify_file_type(file_path)

# print(f"The file is in {file_format} format.")
text = ""
if(file_format == "PDF"):
    doc = fitz.open(file_path)

    for page in doc:
      text = text + str(page.get_text())
#     print(text)
else:
    def read_docx(file_path):
        doc = Document(file_path)
        content = []

        for paragraph in doc.paragraphs:
            content.append(paragraph.text)

        return '\n'.join(content)

    text = read_docx(file_path)
#     print(text)


print(text)

ANAM IQBAL 
Pittsburgh, PA | +1 (646) 207-4431 | anami@andrew.cmu.edu | LinkedIn | Github 
 
EDUCATION 
  
CARNEGIE MELLON UNIVERSITY (CMU)                                                                                                               Pittsburgh, PA 
Master of Information Systems Management: Business Intelligence & Data Analytics (Focus in Data Science)             December 2022        
Relevant Coursework: Introduction to Deep Learning, Big Data & Large Scale Computing, Unstructured Data Analytics, 
                                     Machine Learning for Problem Solving, Interactive Data Science, Distributed Systems 
 
 LAHORE UNIVERSITY OF MANAGEMENT SCIENCES (LUMS) 
 Lahore, Pakistan 
   Bachelor of Science (Honors) - Management Science                                                                                                                 June 2018            
 
SKILLS 
 
Functional:            Machine Learning, Deep Learning, Natural Language Processing (NL

In [68]:
# text = text.strip()

In [69]:
# text = " ".join(text.split())

In [70]:
text

'Ashwini Kalidas Vedula\nEmail ID: ashwini.vedula@nyu.edu | +1 (201)-616-1347 | LinkedIn: linkedin.com/in/ashwini-vedula\nEducation\nNew York University, New York, NY\nSept 2021- May 2023\nCourant Institute of Mathematical Sciences and Stern School of Business\nMaster of Science in Information Systems - 3.6/4\nCourses: Dealing with Data, Data Science for Business : Technical, Database Management Systems, Real Time\nBig Data Analytics, Cloud Computing\nTechnical Skills\nDatabases & Tools : MySQL, SSMS, MS Access, PostgreSQL, PL/SQL, MongoDB\nData Analysis & Visualization Tools : Python, R , SQL, MS Excel, Tableau, Power BI\nCloud Platforms & Big Data Technologies : AWS (Redshift, EC2, S3, EMR, SageMaker, DynamoDB Table), Azure\n(Storage, IoT, Virtual Machines), GCP (BigTable, DataLab), Apache Hadoop, Hive\nExperience\nClark Associates Inc\nPennsylvania, USA\nFinancial Systems and Data Intern\nMay 2022 - Aug 2022\n• Extracted data from varied data sources to automate Summary Income finan

In [31]:
!pip install scikit-learn

In [ ]:
resume_text = text
print(resume_text)

In [62]:
resume_sections = [
    "Contact Information",
    "Professional Summary",
    "Objective",
    "Profile",
    "Summary of Qualifications",
    "Core Competencies",
    "Experience",
    "Work Experience",
    "Employment History",
    "Professional Experience",
    "Internships",
    "Co-op Experience",
    "Education",
    "Academic Background",
    "Educational Qualifications",
    "Certifications",
    "Licenses",
    "Training",
    "Skills",
    "Summary",
    "Technical Skills",
    "Soft Skills",
    "Language Proficiency",
    "Computer Skills",
    "Programming Languages",
    "Tools and Technologies",
    "Projects",
    "Research Experience",
    "Publications",
    "Presentations",
    "Thesis/Dissertation",
    "Awards and Honors",
    "Achievements",
    "Scholarships",
    "Grants",
    "Extracurricular Activities",
    "Leadership Experience",
    "Volunteer Experience",
    "Community Involvement",
    "Professional Affiliations",
    "Memberships",
    "Conferences Attended",
    "Workshops",
    "Seminars",
    "References",
    "Referees",
    "Additional Information",
    "Interests",
    "Awards",
    "Hobbies",
    "Personal Projects",
    "Mission Statement",
    "Career Goals",
    "Professional Development",
    "Courses",
    "Workshops Attended",
    "Online Courses",
    "Webinars",
    "Conventions",
    "Symposiums",
    "Licensure",
    "Professional Memberships",
    "Industry Involvement",
    "Affiliations",
    "Blog",
    "Website",
    "Portfolio",
    "Licenses & Certifications",
    "Leadership and Achievements"
]

resume_sections = [word.lower() for word in resume_sections]
# print()
resume_final_text = resume_text.split('\n')
resume_final_text

splitter="----------------------------------------------------------------------------------------------------"

dict = {}
current_section = ""

for text in resume_final_text:

    if(text.endswith('\n') or text.endswith(':')):
        text = text[:-1]
    elif len(text) >= 2 and text[-1] == '\n' and text[-2] == ':':
        text = text[:-2]

    if(len(text.split()) > 0 and text.split()[-1].lower() in resume_sections):
        print(splitter)
        current_section = text.split()[-1].lower()
    else:
        if(current_section not in dict):
            dict[current_section] = ""
        dict[current_section] += text + "\n"
    print(text)





Ashwini Kalidas Vedula
Email ID: ashwini.vedula@nyu.edu | +1 (201)-616-1347 | LinkedIn: linkedin.com/in/ashwini-vedula
----------------------------------------------------------------------------------------------------
Education
New York University, New York, NY
Sept 2021- May 2023
Courant Institute of Mathematical Sciences and Stern School of Business
Master of Science in Information Systems - 3.6/4
Courses: Dealing with Data, Data Science for Business : Technical, Database Management Systems, Real Time
Big Data Analytics, Cloud Computing
----------------------------------------------------------------------------------------------------
Technical Skills
Databases & Tools : MySQL, SSMS, MS Access, PostgreSQL, PL/SQL, MongoDB
Data Analysis & Visualization Tools : Python, R , SQL, MS Excel, Tableau, Power BI
Cloud Platforms & Big Data Technologies : AWS (Redshift, EC2, S3, EMR, SageMaker, DynamoDB Table), Azure
(Storage, IoT, Virtual Machines), GCP (BigTable, DataLab), Apache Hadoop, H

In [60]:
dict

{'': 'Ashwini Kalidas Vedula Email ID: ashwini.vedula@nyu.edu | +1 (201)-616-1347 | LinkedIn: linkedin.com/in/ashwini-vedula ',
 'education': 'New York University, New York, NY Sept 2021- May 2023 Courant Institute of Mathematical Sciences and Stern School of Business Master of Science in Information Systems - 3.6/4 Courses: Dealing with Data, Data Science for Business : Technical, Database Management Systems, Real Time Big Data Analytics, Cloud Computing ',
 'skills': 'Databases & Tools : MySQL, SSMS, MS Access, PostgreSQL, PL/SQL, MongoDB Data Analysis & Visualization Tools : Python, R , SQL, MS Excel, Tableau, Power BI Cloud Platforms & Big Data Technologies : AWS (Redshift, EC2, S3, EMR, SageMaker, DynamoDB Table), Azure (Storage, IoT, Virtual Machines), GCP (BigTable, DataLab), Apache Hadoop, Hive ',
 'experience': 'Clark Associates Inc Pennsylvania, USA Financial Systems and Data Intern May 2022 - Aug 2022 • Extracted data from varied data sources to automate Summary Income finan

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

df = pd.read_excel("/content/Related Occupations.xlsx")

unique_data_t2 = df['Title'].drop_duplicates()
unique_data_t2

doc = nlp(text)
# print(text)
for ent in doc.ents:
    if(ent.label_ == 'Designation'):
        job_titles = unique_data_t2
        new_job_titles = [
            ent.text
        ]

        job_titles = [title.lower() for title in job_titles]
        new_job_titles = [title.lower() for title in new_job_titles]
        vectorizer = TfidfVectorizer()

        tfidf_matrix = vectorizer.fit_transform(job_titles)

        similar_jobs = {}
        for new_title in new_job_titles:
            new_tfidf = vectorizer.transform([new_title])

            similarities = cosine_similarity(new_tfidf, tfidf_matrix)

            most_similar_index = similarities.argmax()

            similar_jobs[new_title] = job_titles[most_similar_index]

        print("Given Name ...... Standardized Name")
        for new_title, similar_title in similar_jobs.items():
            print(f"{new_title} -> {similar_title}")
            print()


Given Name ...... Standardized Name
senior analyst -> chief executives

Given Name ...... Standardized Name
software engineer -> software developers



In [88]:
print("Different Section parsed from resume are: ")

if "education" in dict:
  print("EDUCATION: ")
  print(dict["education"])


if "experience" in dict:
  print("WORK EXPERIENCE: ")
  print(dict["experience"])
  doc = nlp(dict["experience"])
  for ent in doc.ents:
    if(ent.label_=="Skills"):
      print(ent.text + " ")


if "skills" in dict:
  print("SKILLS: ")
  print(dict["skills"])

Different Section parsed from resume are: 
EDUCATION: 
New York University, New York, NY
Sept 2021- May 2023
Courant Institute of Mathematical Sciences and Stern School of Business
Master of Science in Information Systems - 3.6/4
Courses: Dealing with Data, Data Science for Business : Technical, Database Management Systems, Real Time
Big Data Analytics, Cloud Computing

WORK EXPERIENCE: 
Clark Associates Inc
Pennsylvania, USA
Financial Systems and Data Intern
May 2022 - Aug 2022
• Extracted data from varied data sources to automate Summary Income financial reports(monthly, quarterly and
yearly) for the various subsidiaries of Clark Associates Inc. that reduced manual entry time by 70%.
• Collaborated with cross-functional teams to enhance and optimize existing Power BI reports that helped stakeholders
analyze the financial performance of their business.
Dacapo Brokerage India Private Limited
Mumbai,India
Senior Analyst
Aug 2020 - Jul 2021
• Created visualization of daily top gainer/los